In this notebook, we will build a 3D map of a scene from a small set of images and refine it with the featuremetric optimization. We then localize an image downloaded from the Internet and show the effect of the refinement.

# Setup
We start by defining some output paths: where the intermediate files will be stored.

In [1]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
import os
import time
import sys
import numpy as np
from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect
from PIL import Image, ImageDraw
import pycolmap
from pathlib import Path
#import visualize_model
# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

In [2]:
import torch 
print(torch.__version__)
print(torch.cuda.get_arch_list())

1.9.1+cu111
['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86']


In [3]:
# images = Path('datasets/monarch/')
# outputs = Path('outputs/monarch-demo/')
# sfm_pairs = outputs / 'pairs-sfm.txt'
# loc_pairs = outputs / 'pairs-loc.txt'
# features = outputs / 'features.h5'
# matches = outputs / 'matches.h5'
# raw_dir = outputs / "raw"
# ref_dir = outputs / "ref"
# ''' model location in case of intrinsics locked '''
# ref_dir_locked = outputs / "ref_locked"
# ''' model location in case of intrinsics not locked '''
# ref_dir_not_locked = outputs / "ref_dir_not_locked" 

### Helper functions for relative pose calculations

In [34]:
'''
e_lw => left camera pose in world frame (4 * 4)
e_rw => right camera pose in world frame (4 * 4)
'''
#def calculate_relative_pose(e_lw, e_rw):
def calculate_relative_pose(e_lw: np.ndarray, e_rw: np.ndarray):
#     print(f"Inside the calculate_relative_pose function")
    from scipy.spatial.transform import Rotation
    e_wl = np.linalg.inv(e_lw)
    #print(f"e_wl: {e_wl}")
    #e_rl = e_rw * np.linalg.inv(e_lw) #right camera in the frame of the left camera
    #e_rl = e_rw * e_wl #right camera in the frame of the left camera
    #print(f"e_rl: {e_rl}")
    e_rl = np.dot(e_rw,np.linalg.inv(e_lw))
    print(f"e_rl: \n{e_rl}")
    R = e_rl[:3,:3] #extracting the rotation matrix
    dx = e_rl[0,3]
    dy = e_rl[1,3]
    dz = e_rl[2,3]
    dquat = Rotation.from_matrix(R).as_quat()
    #rel_pose =  [dx, dy] + dquat
    rel_pose = [dx,dy,dz]
    for q in dquat: 
        rel_pose.append(q)
    return rel_pose
    #return [dx,dy]
    #print(f"dx: {dx} dy: {dy} dquat: {dquat}")


def cam_extrinsics(img):
    from read_write_model import qvec2rotmat
    R = qvec2rotmat(img.qvec)
    t = img.tvec.reshape(3,-1)
    #print(f"R: {R} t: {t}")
    R_t = np.concatenate((R,t), axis = 1)
    #R_t = np.vstack([np.array([0,0,0,1]), R_t])
    R_t = np.vstack([R_t, np.array([0,0,0,1])])
    return R_t    #  4 * 4 matrix

def calculate_relative_pose_between(left_idx: int, right_idx: int):
    left_img = sparse_img_dict[left_idx]
    right_img = sparse_img_dict[right_idx]
    e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
    e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
    rel_pose = calculate_relative_pose(e_lw, e_rw)
    return rel_pose


    

### Camera positions WITHOUT Rig Bundle Adjustment

In [5]:
from pathlib import Path
sparse_dir = Path("/home/skumar/ssd/workstation-sfm-setup/output/sparse-reconstruction/front_2023-11-03-11-13-52.svo_200_300/ref_locked")
 
print(f"sparse_dir: {sparse_dir.as_posix()}")
sparse_images = sparse_dir / "images.bin"
sparse_points3D = sparse_dir / "points3D.bin"
sparse_cameras = sparse_dir / "cameras.bin"

sparse_dir: /home/skumar/ssd/workstation-sfm-setup/output/sparse-reconstruction/front_2023-11-03-11-13-52.svo_200_300/ref_locked


In [6]:
import sys
sys.path.append(os.path.expandvars('$HOME/colmap/scripts/python'))
#sys.path.append("/home/skumar/colmap/scripts/python")
from read_write_model import read_images_binary 
sparse_img_dict = read_images_binary(sparse_images)
print(f"{len(sparse_img_dict.keys())} ==> {sparse_img_dict.keys()}")
print(f"min_key: {min(sparse_img_dict.keys())} mx_key: {max(sparse_img_dict.keys())}")

98 ==> dict_keys([98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])
min_key: 1 mx_key: 98


In [29]:
import numpy as np
rel_poses = []
num_images = len(sparse_img_dict.keys())
for idx in range(1, num_images // 2 + 1):
    left_img = sparse_img_dict[idx]
    right_img = sparse_img_dict[idx + 42]
    #print(f"left_img_name: {left_img.name} right_img_name: {right_img.name}")
    e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
    e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
    e_rl = calculate_relative_pose(e_lw, e_rw)
    rel_poses.append(e_rl)

In [8]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.notebook_repr_html', True)
df = pd.DataFrame(rel_poses, columns=['dx', 'dy', 'dz', 'qx' , 'qy', 'qz' , 'qw'])
df.style

,dx,dy,dz,qx,qy,qz,qw
0,0.216536,4.827016,-9.638343,-0.002526,-0.019201,0.001928,0.999811
1,0.137558,4.826466,-9.592770,0.001449,-0.014169,-0.001020,0.999898
2,0.063248,4.933964,-9.634310,0.006003,-0.009061,-0.015930,0.999814
3,0.065474,5.010674,-9.635380,0.009795,-0.006968,-0.013884,0.999831
4,0.110270,5.043970,-9.629129,0.010455,-0.011096,-0.007827,0.999853
5,0.215126,5.017355,-9.631476,0.009604,-0.015146,-0.012103,0.999766
6,0.330663,4.974886,-9.642736,0.006991,-0.023721,-0.009380,0.999650
7,-0.097640,-0.818630,1.581503,0.001288,0.001801,0.002038,0.999995
8,-0.098471,-0.797500,1.605028,-0.001660,-0.000117,0.002499,0.999995
9,-0.053826,-0.807948,1.628691,-0.002980,-0.006076,0.010846,0.999918


### Camera poses with Rig Bundle Adjustment

In [13]:
rig_ba_sparse_dir = Path("/home/skumar/ssd/workstation-sfm-setup/output/rig-bundle-adjustment/front_2023-11-03-11-13-52.svo_200_300/")

rig_ba_sparse_images = rig_ba_sparse_dir / "images.bin"
rig_ba_sparse_points3D = rig_ba_sparse_dir / "points3D.bin"
rig_ba_sparse_cameras = rig_ba_sparse_dir / "cameras.bin"

In [14]:
import sys
#sys.path.append("/home/skumar/colmap/scripts/python")
sys.path.append(os.path.expandvars('$HOME/colmap/scripts/python'))
from read_write_model import read_images_binary 
rig_ba_sparse_img_dict = read_images_binary(rig_ba_sparse_images)
print(f"{len(rig_ba_sparse_img_dict.keys())} => {rig_ba_sparse_img_dict.keys()}")

98 => dict_keys([59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 13, 12, 11, 10, 9, 8, 7, 6, 5, 70, 71, 72, 73, 14, 74, 15, 75, 16, 76, 17, 77, 18, 78, 19, 79, 20, 80, 21, 81, 22, 82, 23, 83, 24, 84, 25, 85, 26, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 29, 87, 28, 86, 27, 69, 68, 67, 66, 65, 64, 63, 4, 62, 3, 61, 2, 60, 1])


In [21]:
#  for k, v in rig_ba_sparse_img_dict.items():
#         print(f"{k} {v.name}")

print(rig_ba_sparse_img_dict.keys())

dict_keys([59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 13, 12, 11, 10, 9, 8, 7, 6, 5, 70, 71, 72, 73, 14, 74, 15, 75, 16, 76, 17, 77, 18, 78, 19, 79, 20, 80, 21, 81, 22, 82, 23, 83, 24, 84, 25, 85, 26, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 29, 87, 28, 86, 27, 69, 68, 67, 66, 65, 64, 63, 4, 62, 3, 61, 2, 60, 1])


In [17]:
# import numpy as np
# rig_ba_rel_poses = []
# num_images = len(rig_ba_sparse_img_dict.keys())
# for idx in range(1, num_images // 2):
#     left_img = rig_ba_sparse_img_dict[idx]
#     right_img = rig_ba_sparse_img_dict[idx + num_images // 2]
#     if idx < 5:
#         print(f"left_img_name: {left_img.name} right_img_name: {right_img.name}")
#     e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
#     e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
#     rel_pose = calculate_relative_pose(e_lw, e_rw)
#     rig_ba_rel_poses.append(rel_pose)

left_img_name: left/frame_202_.jpg right_img_name: right/frame_202_.jpg
left_img_name: left/frame_204_.jpg right_img_name: right/frame_204_.jpg
left_img_name: left/frame_206_.jpg right_img_name: right/frame_206_.jpg
left_img_name: left/frame_208_.jpg right_img_name: right/frame_208_.jpg


In [35]:
import numpy as np
rig_ba_rel_poses = []
num_images = len(rig_ba_sparse_img_dict.keys())
for idx in range(1, num_images // 2):
    left_img = rig_ba_sparse_img_dict[idx]
    right_img = rig_ba_sparse_img_dict[idx + num_images // 2]
    e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
    e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
    print(f"e_lw: \n{e_lw}")
    print(f"e_rw: \n{e_rw}\n")
    rel_pose = calculate_relative_pose(e_lw, e_rw)
    rig_ba_rel_poses.append(rel_pose)
#     print(f"rel_pose: {rel_pose}")
    if idx > 5: 
        break
    
    

e_lw: 
[[-9.99484844e-01 -1.20282614e-02  2.97551136e-02  1.15468604e-01]
 [ 1.19752990e-02 -9.99926377e-01 -1.95751118e-03 -3.06252757e+00]
 [ 2.97764684e-02 -1.60017638e-03  9.99555302e-01  5.90440520e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
e_rw: 
[[-9.99520616e-01 -1.22073446e-02  2.84520629e-02 -2.27204570e-02]
 [ 1.21549138e-02 -9.99924096e-01 -2.01500646e-03 -3.06273211e+00]
 [ 2.84745012e-02 -1.66820813e-03  9.99593127e-01  5.90260507e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

e_rl: 
[[ 9.99999134e-01  1.81192463e-04 -1.30324979e-03 -1.29939139e-01]
 [-1.81260549e-04  9.99999982e-01 -5.21250650e-05  1.24100290e-04]
 [ 1.30324033e-03  5.23612476e-05  9.99999149e-01 -1.78523956e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
e_lw: 
[[-9.99610970e-01 -8.14528159e-03  2.66751391e-02  1.19591382e-01]
 [ 8.22636506e-03 -9.99961866e-01  2.93133508e-03 -2.90016977e+00]
 [ 2.66502453e-02  3.14963413e

In [24]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.notebook_repr_html', True)
df = pd.DataFrame(rig_ba_rel_poses, columns=['dx', 'dy', 'dz', 'qx' , 'qy', 'qz' , 'qw'])
df.style

,dx,dy,dz,qx,qy,qz,qw
0,-0.129939,0.000124,-0.001785,0.000026,-0.000652,-0.000091,1.000000
1,-0.129939,0.000124,-0.001785,0.000026,-0.000652,-0.000091,1.000000
2,-0.129939,0.000124,-0.001785,0.000026,-0.000652,-0.000091,1.000000
3,-0.129939,0.000124,-0.001785,0.000026,-0.000652,-0.000091,1.000000
4,-0.129939,0.000124,-0.001785,0.000026,-0.000652,-0.000091,1.000000
5,-0.129939,0.000124,-0.001785,0.000026,-0.000652,-0.000091,1.000000
